In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from mpl_toolkits.mplot3d import Axes3D
#vom 8.11.2025
#based on Application of a dynamic method of minimisation in the study of reaction surfaces by Colin M. Smith

In [2]:
# saddle at 0,0,0
def f(x,y,z):
    return np.sin(x) * y + np.sin(y) * z +   + np.sin(z) * x

def dfx(x,y,z):
    return y*np.cos(x) + np.sin(z)

def dfy(x,y,z):
    return np.sin(x) + z*np.cos(y)

def dfz(x,y,z):
    return np.sin(y) + x*np.cos(z)

def ddfxx(x,y,z):
    return -y * np.sin(x)

def ddfyy(x,y,z):
    return -z * np.sin(y)

def ddfzz(x,y,z):
    return -x * np.sin(z)

def ddfxy(x,y,z):
    return np.cos(x)

def ddfxz(x,y,z):
    return np.cos(z)

def ddfyx(x,y,z):
    return np.cos(x)

def ddfyz(x,y,z):
    return np.cos(y)

def ddfzx(x,y,z):
    return np.cos(z)

def ddfzy(x,y,z):
    return np.cos(y)
    


In [3]:
def gradient(x,y,z):
    g = [dfx(x,y,z),dfy(x,y,z), dfz(x,y,z)]
    return np.array(g)
def hessian(x,y,z):
    H = [[ddfxx(x,y,z),ddfxy(x,y,z), ddfxz(x,y,z)],
        [ddfyx(x,y,z),ddfyy(x,y,z),ddfyz(x,y,z)],
        [ddfzx(x,y,z),ddfzy(x,y,z),ddfzz(x,y,z)]]
    return np.array(H)

g = gradient(0.5,1.0,0.5)
H = hessian(0.5,1.0,0.5)
eigs = np.linalg.eigvals(H)
print(eigs)

print(gradient(4,-4,4))



[ 1.15597243 -1.42815513 -0.86769111]
[ 1.85777199 -3.37137698 -1.85777199]


In [4]:
#gradient decend
def findSaddle(point,n, step=0.01,iterations = 1000, etol = 1e-6):
    x,y,z = point
    H = hessian(x,y,z)
    u, U = np.linalg.eig(H)
    path = [(x,y,z)]

    for i in range (iterations):
        g = gradient(x,y,z)
        if np.linalg.norm(g) < etol:
            #print(f"Converged at iteration {i}")
            break
        H = hessian(x,y,z)
        u, U = np.linalg.eigh(H)
        #idx = np.argmin(u)
        
        idx = np.argsort(u)[:n]
        #idx = np.where(u < 0)[0]
        #find principal gradients
        g_p = U.T @ g
    
        #reflect at smallets eigenvalue
        g_p[idx] = -g_p[idx]
        
        #transform back 
        g = U @ g_p

        #gradient step
        x = x - step * g[0]
        y = y - step * g[1]
        z = z - step * g[2]
        
        path.append((x, y, z))

    path = np.array(path)
    saddleType = len(np.where(u < 0)[0])
    
    #no saddle found
    if np.linalg.norm(g) > etol:
        return path, -1
         
    return path, saddleType

In [5]:
#Newton step
def findSaddle(point,n, step=0.01,iterations = 1000, etol = 1e-6):
    lam = step
    x,y,z = point
    H = hessian(x,y,z)
    u, U = np.linalg.eig(H)
    path = [(x,y,z)]

    for i in range (iterations):
        g = gradient(x,y,z)
        if np.linalg.norm(g) < etol:
            #print(f"Converged at iteration {i}")
            break
        H = hessian(x,y,z)
        u, U = np.linalg.eigh(H)
        #idx = np.argmin(u)
        
        idx = np.argsort(u)[:n]
        #idx = np.where(u < 0)[0]
        #find principal gradients
        g_p = U.T @ g
    
        #reflect at smallets eigenvalue
        g_p[idx] = -g_p[idx]
        
        #transform back 
        g = U @ g_p
    
        step = np.linalg.solve(H, g)

        #newton step
        x = x -  0.1 * step[0]
        y = y -  0.1 * step[1]
        z = z -  0.1 * step[2]
    
        path.append((x, y, z))

    path = np.array(path)
    saddleType = len(np.where(u < 0)[0])
    
    #no saddle found
    if np.linalg.norm(g) > etol:
        return path, -1
         
    return path, saddleType

In [6]:
def findExtremeValues(size=5,iterations = 100,step=0.01,etol=1e-6):
    minima = []
    saddle1 = []
    saddle2 = []
    maxima = []
    extremes = [minima,saddle1,saddle2,maxima]
    for n in range(4):
        for i in range(iterations):
            x, y, z = np.random.uniform(-size, size, 3)
            path, saddleType = findSaddle((x,y,z),n,step=step,etol=etol)
            if saddleType>=0:
                extremes[saddleType].append(path[-1])
    for n in range(4):
        print(len(extremes[n]))
        extremes[n] = np.unique(np.array(extremes[n]).round(decimals=2), axis=0)
        print(len(extremes[n]))
    print(extremes)
    return extremes
extremes = findExtremeValues(5,200,step=0.1)

6
4
84
25
102
28
12
6
[array([[-5.2 , -2.1 ,  1.74],
       [ 1.74, -5.2 , -2.1 ],
       [ 4.91,  4.91,  4.91],
       [ 5.2 ,  2.1 , -1.74]]), array([[-29.81, -26.76,  17.25],
       [ -7.65,   4.93,   4.58],
       [ -4.96,  -3.96,   1.42],
       [ -4.53,  -5.48,  -1.41],
       [ -4.13,  -1.76,   4.47],
       [ -2.57,   1.11,   1.21],
       [ -1.76,   4.47,  -4.13],
       [ -1.41,  -4.53,  -5.48],
       [ -1.21,   2.57,  -1.11],
       [ -1.11,  -1.21,   2.57],
       [  1.11,   1.21,  -2.57],
       [  1.21,  -2.57,   1.11],
       [  1.22,  -1.71,   6.91],
       [  1.41,   4.53,   5.48],
       [  1.42,  -4.96,  -3.96],
       [  1.68,  -7.72,  -7.22],
       [  1.76,  -4.47,   4.13],
       [  2.57,  -1.11,  -1.21],
       [  4.47,  -4.13,  -1.76],
       [  4.5 ,  -4.59,  -8.08],
       [  4.53,   5.48,   1.41],
       [  4.63,  11.78,   1.42],
       [  4.84,   7.63,   4.51],
       [  4.96,   3.96,  -1.42],
       [  5.48,   1.41,   4.53]]), array([[ -5.71,   1.18,  -1.

def createLine(points, color='grey'):
    lines=[]
    for startpoint in points:
        endpoints = []
        for n in range(4):
            for j in range(10): 
                x, y, z = startpoint + np.random.uniform(-0.1, 0.1, 3)
                path, saddleType = findSaddle((x,y,z),n,iterations=1000,etol=1e-3)
                if saddleType >= 0:
                    endpoints.append(path[-1])

        endpoints = np.unique(np.array(endpoints).round(decimals=2), axis=0)

        for endpoint in endpoints:
            line = go.Scatter3d(
                    x=[startpoint[0],endpoint[0]],
                    y=[startpoint[1],endpoint[1]],
                    z=[startpoint[2],endpoint[2]],
                    mode='lines',
                    line=dict(color='grey', width=1),
                    opacity=0.5,
                    showlegend=False
                    )
            lines.append(line)
        return lines
    

lines=[]
for i in range(4):
    lines = lines + createLine(extremes[i])

In [7]:
def showConnectivity(extremes,iterations = 20,step=0.1):
    lines=[]

    for i in range(4):
        for extreme in extremes[i]:
            endpoints = []
            for n in range(4):
                #same type
                if n == i:
                    continue
                for j in range(iterations): 
                    x, y, z = extreme + np.random.uniform(-0.1, 0.1, 3)
                    path, saddleType = findSaddle((x,y,z),n,iterations=1000,step=step,etol=1e-3)
                    if saddleType >= 0:
                        endpoints.append(path[-1])

            endpoints = np.unique(np.array(endpoints).round(decimals=2), axis=0)

        
            for point in endpoints:
                line = go.Scatter3d(
                        x=[extreme[0],point[0]],
                        y=[extreme[1],point[1]],
                        z=[extreme[2],point[2]],
                        mode='lines',
                        line=dict(color='grey', width=1),
                        opacity=0.5,
                        showlegend=False
                        )
                lines.append(line)
    return lines
                
lines = showConnectivity(extremes,2,step=0.1)


In [ ]:
def getNeighborhood(point,iterations = 100):
    neighbors = []
    for n in range(4):
            for j in range(iterations):
                x, y, z = point + np.random.uniform(-0.1, 0.1, 3)
                path, saddleType = findSaddle((x,y,z),n,iterations=3000,etol=1e-3)
                if saddleType >= 0:
                    neighbors.append(path[-1])
                    
    return np.unique(np.array(neighbors).round(decimals=2), axis=0)
                    

In [ ]:
def findNeighborsOfType(point,saddleType, iterations = 100):
    neighbors = []
    for j in range(iterations):
        x, y, z = point + np.random.uniform(-0.1, 0.1, 3)
        path, saddleType = findSaddle((x,y,z),saddleType,iterations=3000,etol=1e-3)
        if saddleType >= 0:
            neighbors.append(path[-1])
                    
    return np.unique(np.array(neighbors).round(decimals=2), axis=0)


In [ ]:

start=np.array([4.91,4.91,4.91])
sad1 = np.array([4.53,5.48,1.41])
sad2 = np.array([4.58,7.25,1.75])
end = np.array([4.83,8.46,-1.74])
fpath1 = f(*start) + f(*sad1) + f(*end)
fpath2 = f(*start) + f(*sad2) + f(*end)

dist1 = np.linalg.norm(start - sad1) + np.linalg.norm(end - sad1)
dist2 = np.linalg.norm(start - sad2) + np.linalg.norm(end - sad2)
print(fpath1, fpath2)
print(dist1,dist2)
print(fpath1 + dist1, fpath2 + dist2)

In [ ]:
def colorNeighborhood(point, color="purple",iterations = 100):
    lines = []
    for n in range(4):
        for j in range(iterations):
            x, y, z = point + np.random.uniform(-0.1, 0.1, 3)
            path, saddleType = findSaddle((x,y,z),n,iterations=3000,etol=1e-3)
            if saddleType >= 0:
                line = go.Scatter3d(
                x=[point[0],path[-1,0]],
                y=[point[1],path[-1,1]],
                z=[point[2],path[-1,2]],
                mode='lines',
                line=dict(color=color, width=1),
                opacity=0.5,
                showlegend=False
                )
                lines.append(line)
    return lines
lines += colorNeighborhood([0,0,0],color="red",iterations=3)

In [11]:
def showConnectivityOfType(points,saddle_type, color="teal",iterations = 100):
    lines = []
    for point in points:
        endpoints = []
        for j in range(iterations): 
            x, y, z = point + np.random.uniform(-0.1, 0.1, 3)
            path, saddleType = findSaddle((x,y,z),saddle_type,iterations=1000,etol=1e-3)
            if saddleType >= 0:
                endpoints.append(path[-1])

        endpoints = np.unique(np.array(endpoints).round(decimals=2), axis=0)

        
        for endpoint in endpoints:
            line = go.Scatter3d(
                x=[point[0],endpoint[0]],
                y=[point[1],endpoint[1]],
                z=[point[2],endpoint[2]],
                mode='lines',
                line=dict(color=color, width=1),
                opacity=0.5,
                showlegend=False
                        )
            lines.append(line)
    return lines
lines = showConnectivityOfType(extremes[1],0)
#lines = showConnectivityOfType(extremes[2],0,color="red")
#lines = showConnectivityOfType(extremes[3],0,color="black")

In [ ]:
#connectivity of Type1 saddle points
lines = showConnectivityOfType(extremes[1],0,color="red")
lines += showConnectivityOfType(extremes[1],2,color="blue")
lines += showConnectivityOfType(extremes[1],3,color="yellow")


In [8]:
#Visualize
#Extreme values
minima = extremes[0]
minima_points = go.Scatter3d(
    x=minima[:, 0],
    y=minima[:, 1],
    z=minima[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color='red',
        opacity=1.0,
        symbol='circle'
    ),
    name='Minima'
)
saddle1 = extremes[1]
saddle1_points = go.Scatter3d(
    x=saddle1[:, 0],
    y=saddle1[:, 1],
    z=saddle1[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color='blue',
        opacity=1.0,
        symbol='circle'
    ),
    name='SaddleType1'
)
saddle2 = extremes[2]
saddle2_points = go.Scatter3d(
    x=saddle2[:, 0],
    y=saddle2[:, 1],
    z=saddle2[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color='green',
        opacity=1.0,
        symbol='circle'
    ),
    name='SaddleType2'
)
maxima = extremes[3]
maxima_points = go.Scatter3d(
    x=maxima[:, 0],
    y=maxima[:, 1],
    z=maxima[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color='yellow',
        opacity=1.0,
        symbol='circle'
    ),
    name='Maxima'
)



# Combine figure
fig = go.Figure(data=[minima_points,saddle1_points,saddle2_points,maxima_points]+lines)

fig.update_layout(
    title='Connectivity of Extreme Values',
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z'
    ),
    template='plotly_white'
)

fig.show(renderer="browser")
